In [1]:
#constraint programming (CP)
#Пример задач - составление расписания
#То есть задачи которые ищут подходящее но не всегда оптимальное решение

# Simple example

- Three variables, x, y, and z, each of which can take on the values: 0, 1, or 2<br>
- One constraint: x ≠ y

In [2]:
model.NewIntVar?

Object `model.NewIntVar` not found.


In [3]:
from ortools.sat.python import cp_model

# Creates the model.
model = cp_model.CpModel()

# Creates the variables.
num_vals = 3
x = model.NewIntVar(0, num_vals - 1, 'x') #lower bound, upper bound, name
y = model.NewIntVar(0, num_vals - 1, 'y')
z = model.NewIntVar(0, num_vals - 1, 'z')

# Creates the constraints.
model.Add(x != y)

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))
    print('z = %i' % solver.Value(z))

x = 1
y = 0
z = 0


# Finding all solutions

In [5]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count

In [6]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([x, y, z])
status = solver.SearchForAllSolutions(model, solution_printer)

x=1 y=2 z=0 
x=1 y=0 z=0 
x=2 y=0 z=0 
x=2 y=1 z=0 
x=2 y=1 z=1 
x=2 y=0 z=1 
x=1 y=0 z=1 
x=1 y=2 z=1 
x=1 y=2 z=2 
x=1 y=0 z=2 
x=2 y=0 z=2 
x=2 y=1 z=2 
x=0 y=1 z=2 
x=0 y=1 z=1 
x=0 y=1 z=0 
x=0 y=2 z=0 
x=0 y=2 z=1 
x=0 y=2 z=2 


# Solving an Optimization Problem

we'll show how to find an optimal solution<br>

Maximize 2x + 2y + 3z subject to the following constraints:<br>
x + 7⁄2 y + 3⁄2 z	≤	25<br>
3x - 5y + 7z	≤	45<br>
5x + 2y - 6z	≤	37<br>
x, y, z	≥	0<br>
x, y, z integers<br>

In order to increase computational speed, the CP-SAT solver works over the integers. This means all constraints and the objective must have integer coefficients. In the above example, the first constraint doesn't meet this condition. To solve the problem, you must first transform the constraint by multiplying it by a sufficiently large integer to convert all the coefficients to integers.

In [7]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()
var_upper_bound = max(50, 45, 37)
x = model.NewIntVar(0, var_upper_bound, 'x')
y = model.NewIntVar(0, var_upper_bound, 'y')
z = model.NewIntVar(0, var_upper_bound, 'z')

model.Add(2*x + 7*y + 3*z <= 50)
model.Add(3*x - 5*y + 7*z <= 45)
model.Add(5*x + 2*y - 6*z <= 37)

model.Maximize(2*x + 2*y + 3*z)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('Maximum of objective function: %i' % solver.ObjectiveValue())
    print()
    print('x value: ', solver.Value(x))
    print('y value: ', solver.Value(y))
    print('z value: ', solver.Value(z))

Maximum of objective function: 35

x value:  7
y value:  3
z value:  5


# Setting solver limits

In [ ]:
solver.parameters.max_time_in_seconds = 10.0

In [8]:
#Stopping a search after a specified number of solutions
from ortools.sat.python import cp_model


class VarArraySolutionPrinterWithLimit(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.__solution_limit = limit

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()
        if self.__solution_count >= self.__solution_limit:
            print('Stop search after %i solutions' % self.__solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self.__solution_count


def StopAfterNSolutionsSampleSat():
    """Showcases calling the solver to search for small number of solutions."""
    # Creates the model.
    model = cp_model.CpModel()
    # Creates the variables.
    num_vals = 3
    x = model.NewIntVar(0, num_vals - 1, 'x')
    y = model.NewIntVar(0, num_vals - 1, 'y')
    z = model.NewIntVar(0, num_vals - 1, 'z')

    # Create a solver and solve.
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinterWithLimit([x, y, z], 5)
    status = solver.SearchForAllSolutions(model, solution_printer)
    print('Status = %s' % solver.StatusName(status))
    print('Number of solutions found: %i' % solution_printer.solution_count())
    assert solution_printer.solution_count() == 5

StopAfterNSolutionsSampleSat()

x=0 y=0 z=0 
x=0 y=1 z=0 
x=0 y=1 z=1 
x=0 y=0 z=1 
x=0 y=2 z=1 
Stop search after 5 solutions
Status = FEASIBLE
Number of solutions found: 5
